In [2]:
# Importing essential libraries

import pandas as pd
import numpy as np
import time

start = time.time()

### Loading the data

In [3]:
# Loading data 
%%time
df = pd.read_csv('../Data/raw_data.csv.gz', compression='gzip')
len(df)

CPU times: total: 1min 11s
Wall time: 6min 37s


55967041

In [4]:
df

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-01-01 00:00:00 UTC,view,1005073,2232732093077520756,construction.tools.light,samsung,1130.02,519698804,69b5d72f-fd6e-4fed-aa23-1286b2ca89a0
1,2020-01-01 00:00:01 UTC,view,1005192,2232732093077520756,construction.tools.light,meizu,205.67,527767423,7f596032-ccbf-4643-9bad-e36a209512b4
2,2020-01-01 00:00:01 UTC,view,100063693,2053013552427434207,apparel.shirt,turtle,136.43,519046195,d1e2f343-84bb-49bd-b13d-ca0f1ed9910e
3,2020-01-01 00:00:01 UTC,view,5100816,2232732103831716449,apparel.shoes,xiaomi,29.95,518269232,0444841c-38ef-410c-b11f-7b35ea4e5991
4,2020-01-01 00:00:02 UTC,view,100014325,2232732103294845523,apparel.shoes.step_ins,intel,167.20,587748686,31b7d4cf-dfac-4895-9927-90fa3254f860
...,...,...,...,...,...,...,...,...,...
55967036,2020-01-31 23:59:59 UTC,view,5100833,2232732103831716449,apparel.shoes,xiaomi,33.72,517548219,37af3593-a086-4ad8-b6e5-8f87370d91e3
55967037,2020-01-31 23:59:59 UTC,view,1005239,2232732093077520756,construction.tools.light,xiaomi,228.83,512985520,4d185c67-1425-4f89-904e-bd156c840702
55967038,2020-01-31 23:59:59 UTC,view,100043202,2053013556521075159,apparel.shoes,respect,47.36,513043451,4d614d05-3b0c-4504-a75e-9e09715f737f
55967039,2020-01-31 23:59:59 UTC,view,2900565,2232732102103663163,furniture.bedroom.blanket,midea,51.30,581011571,a60355b0-eb87-4d42-a9ca-d4032c5b1ef5


In [5]:
df.isnull().sum()

event_time             0
event_type             0
product_id             0
category_id            0
category_code    5044890
brand            6540255
price                  0
user_id                0
user_session          19
dtype: int64

In [6]:
df.memory_usage(index=False, deep=True)

event_time       4477363280
event_type       3417329529
product_id        447736328
category_id       447736328
category_code    4203242491
brand            3324321930
price             447736328
user_id           447736328
user_session     5204933654
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55967041 entries, 0 to 55967040
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 3.8+ GB


Memory Usage was high as expected for 55M+ rows. To have a production friendly code, we proceed to optimise and lower memory usage by dataframe as much as possible.

In [8]:
# Validating that certain numerical columns fit within 32-bit limits before type conversion

print(df['user_id'].max()/(2**32))
print(df['product_id'].max()/(2**32))
print(df['price'].max()/(2**32))
print(df['category_id'].max()/(2**32))

assert df['product_id'].max()/(2**32) < 1
assert df['user_id'].max()/(2**32) < 1
assert df['price'].max()/(2**32) < 1

0.1417524558492005
0.023306243354454637
5.993223749101162e-07
520650863.3183595


In [9]:
df

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-01-01 00:00:00 UTC,view,1005073,2232732093077520756,construction.tools.light,samsung,1130.02,519698804,69b5d72f-fd6e-4fed-aa23-1286b2ca89a0
1,2020-01-01 00:00:01 UTC,view,1005192,2232732093077520756,construction.tools.light,meizu,205.67,527767423,7f596032-ccbf-4643-9bad-e36a209512b4
2,2020-01-01 00:00:01 UTC,view,100063693,2053013552427434207,apparel.shirt,turtle,136.43,519046195,d1e2f343-84bb-49bd-b13d-ca0f1ed9910e
3,2020-01-01 00:00:01 UTC,view,5100816,2232732103831716449,apparel.shoes,xiaomi,29.95,518269232,0444841c-38ef-410c-b11f-7b35ea4e5991
4,2020-01-01 00:00:02 UTC,view,100014325,2232732103294845523,apparel.shoes.step_ins,intel,167.20,587748686,31b7d4cf-dfac-4895-9927-90fa3254f860
...,...,...,...,...,...,...,...,...,...
55967036,2020-01-31 23:59:59 UTC,view,5100833,2232732103831716449,apparel.shoes,xiaomi,33.72,517548219,37af3593-a086-4ad8-b6e5-8f87370d91e3
55967037,2020-01-31 23:59:59 UTC,view,1005239,2232732093077520756,construction.tools.light,xiaomi,228.83,512985520,4d185c67-1425-4f89-904e-bd156c840702
55967038,2020-01-31 23:59:59 UTC,view,100043202,2053013556521075159,apparel.shoes,respect,47.36,513043451,4d614d05-3b0c-4504-a75e-9e09715f737f
55967039,2020-01-31 23:59:59 UTC,view,2900565,2232732102103663163,furniture.bedroom.blanket,midea,51.30,581011571,a60355b0-eb87-4d42-a9ca-d4032c5b1ef5


In [10]:

# Optimizing memory usage by converting data types 
d = {'product_id': 'int32', 'user_id': 'int32', 'event_type': 'category', 'price':'float32'}
df = df.astype(d)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55967041 entries, 0 to 55967040
Data columns (total 9 columns):
 #   Column         Dtype   
---  ------         -----   
 0   event_time     object  
 1   event_type     category
 2   product_id     int32   
 3   category_id    int64   
 4   category_code  object  
 5   brand          object  
 6   price          float32 
 7   user_id        int32   
 8   user_session   object  
dtypes: category(1), float32(1), int32(2), int64(1), object(4)
memory usage: 2.8+ GB


In [12]:
df.memory_usage(index=False, deep=True)

event_time       4477363280
event_type         55967336
product_id        223868164
category_id       447736328
category_code    4203242491
brand            3324321930
price             223868164
user_id           223868164
user_session     5204933654
dtype: int64

In [13]:
# df = pd.read_csv('../Data/raw_data.csv.gz', compression='gzip', dtype = {'product_id': 'int32', 'user_id': 'int32', 'event_type': 'category', 'price':'float32'})
# df.memory_usage(index=False, deep=True)

In [14]:
df.to_parquet('../Data/optimised_raw_data.parquet',index=False)

In [15]:
print("Process Took: {}mins".format( round((time.time()-start)/60, 2) ))

Process Took: 20.38mins


In [16]:
%%time
new_df = pd.read_parquet('../Data/optimised_raw_data.parquet')
len(new_df)

CPU times: total: 50.1 s
Wall time: 1min 41s


55967041

In [17]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55967041 entries, 0 to 55967040
Data columns (total 9 columns):
 #   Column         Dtype   
---  ------         -----   
 0   event_time     object  
 1   event_type     category
 2   product_id     int32   
 3   category_id    int64   
 4   category_code  object  
 5   brand          object  
 6   price          float32 
 7   user_id        int32   
 8   user_session   object  
dtypes: category(1), float32(1), int32(2), int64(1), object(4)
memory usage: 2.8+ GB


In [18]:
!du -sh ../Data/*

'du' is not recognized as an internal or external command,
operable program or batch file.
